In [1]:
import yaml
import pathlib

import matplotlib.pyplot as plt
from  matplotlib.colors import ListedColormap

import geopandas as gpd
import pandas as pd
from shapely import box
import rasterio as rio

import numpy as np
import os
import fancy_plots as vis

from joblib import Parallel, delayed

In [ ]:
cwd = pathlib.Path(os.getcwd())
with open(cwd / 'config_tyndall.yaml') as stream:
    config = yaml.safe_load(stream)

config['Directories']['work_dir'] = pathlib.Path(config['Directories']['work_dir'])

for key, value in config['Directories'].items():
    if key == 'work_dir' or key == 'color_to_phase_dicts' or key == 'geom_dir':
        continue
    else:
        config['Directories'][key] = config['Directories']['work_dir'] / value
        
# Manual delineations
ais_cci = gpd.read_file(config['Directories']['geom_dir'])
measures = gpd.read_file(config['Directories']['reference_manual_GL'])
radarsat_mosaic = config['Directories']['plot_resources'] / 'satellite_backdrop' / 'amm125m_v2_200m.tif'
selected_rois = gpd.read_file(cwd / 'rois/selected_glaciers.shp')
basemap = gpd.read_file(config['Directories']['plot_resources'] / 'basemap')
cci_area = gpd.GeoDataFrame(geometry = [box(*ais_cci.total_bounds)], crs = 'EPSG:3031')
basemap = basemap.clip(cci_area)
zoom_ins = gpd.read_file(cwd / 'rois/zoom_ins.geojson')

Dict of interferograms used in the figures

In [ ]:
samples = {'Getz_1': 'SEN_HH_069_D_R_01_20171217110531_20171223110448_20171229110530________________000389',
           'Getz_2': 'SEN_HH_054_D_R_01_20171216102441_20171222102359_20171228102441________________000388',
           'Abbot': 'SEN_HH_068_D_R_02_20170906092558_20170912092641_20170918092559________________000328',
           'Cabinet_Inlet': 'SEN_HH_038_D_R_06_20160927080209_20161003080127_20161009080209________________000151',
           'LarsenC': 'SEN_HH_038_D_R_07_20170425080150_20170501080232_20170507080151________________000296',
           'Totten': 'SEN_HH_099_A_R_05_20171002123517_20171008123559_20171014123517________________000314',
           'Amery': 'SEN_HH_003_D_R_06_20170329222716_20170410222716_20170422222717________________000284',
           'Lazarevisen': 'SEN_HH_118_A_R_01_20201210195030_20201216195112_20201222195030________________001397',
           'George_VI': 'ERS_VV_491_A_R____19960218052247_19960219052248_______________________________000148',
           'Dickey': 'Dickey_3031_100.tif',
           'Nimrod': 'Nimrod_3031.tif',
           'Crary': 'Crary_3031_100.tif',
           'Siple': 'TSX_HH_053_D_L____20170227143851_20170310143851_20170321143852________________000242',
           'Ketchum_Glacier': 'ERS_VV_051_D_R____19951109113159_19951110113201_______________________________000120'}

Read DInSAR phase for the samples

In [ ]:
getz1_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['Getz_1'] + '.tif')
getz2_features = config['Directories']['plot_resources'] / 'test_interferograms'  / (samples['Getz_2'] + '.tif')
abbot_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['Abbot'] + '.tif')
cabinet_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['Cabinet_Inlet'] + '.tif')
larsenc_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['LarsenC'] + '.tif')
totten_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['Totten'] + '.tif')
amery_features = config['Directories']['plot_resources'] / 'test_interferograms'  / (samples['Amery'] + '.tif')
lazarevisen_features = config['Directories']['plot_resources'] / 'test_interferograms' / (samples['Lazarevisen'] + '.tif')
george_vi = config['Directories']['plot_resources'] / 'test_interferograms'  / (samples['George_VI'] + '.tif')
crary = config['Directories']['plot_resources'] / 'test_interferograms'/ (samples['Crary'])
nimrod = config['Directories']['plot_resources'] / 'test_interferograms'/ (samples['Nimrod'])
dickey = config['Directories']['plot_resources'] / 'test_interferograms'/ (samples['Dickey'])
siple_features = config['Directories']['downsampled_dinsar'] / (samples['Siple'] + '.tif')

In [ ]:
crs_4326 = vis.ccrs.PlateCarree()
crs_3031 = vis.ccrs.SouthPolarStereo()

Fig 02 Features stack

In [ ]:
fig, axs = plt.subplots(figsize = (10, 5), dpi = 300, nrows = 2, ncols = 4, subplot_kw = {'projection': crs_3031}, tight_layout = True)

with rio.open(amery_features) as tif:
    features = tif.read()

titles = np.array(['Real', 'Imaginary', 'Pseudo coherence', 'Phase', 'DEM', 'Ice velocity easting', 'Ice velocity northing', 'Differential tide level'])
cmaps = np.array(['hsv', 'hsv', 'gray', 'hsv', 'terrain', 'RdBu_r', 'RdBu_r', 'Blues'])
labels = np.array(['', '', '', 'radians', 'm', 'm/d', 'm/d', 'm'])

ice_vel_max =  features[5:7, :].max()
ice_vel_min = -1 * ice_vel_max
lims = [(None, None), (ice_vel_min, ice_vel_max), (ice_vel_min, ice_vel_max), (None, None)]

for channel, (title, cmap, label) in enumerate(zip(titles[:4], cmaps[:4], labels[:4])):
    axs[0, channel].set_facecolor('white')
    axs[0, channel].set_title(title)

    feature = features[channel, : , :]
    if channel != 2:
        feature[feature == 0] = np.nan
    
    plot = axs[0, channel].imshow(feature, cmap = cmap, interpolation = 'None', transform = crs_3031)

    cbar = fig.colorbar(plot, ax = axs[0, channel], label = label, fraction = 0.046, pad = 0.04)
    cbar.set_ticks(ticks = [feature.min(), feature.max()])
    axs[0, channel].axis('off')

cbar.set_ticks(ticks = [-3.14, 3.14], )

for channel, (title, cmap, label) in enumerate(zip(titles[4:], cmaps[4:], labels[4:])):
    axs[1, channel].set_facecolor('white')
    axs[1, channel].set_title(title)

    lim = lims[channel]
    plot = axs[1, channel].imshow(features[channel + 4, :, :], cmap = cmap, interpolation = 'None', transform = crs_3031, vmin = lim[0], vmax = lim[1])
    fig.colorbar(plot, ax = axs[1, channel], label = label, fraction = 0.046, pad = 0.04)
    axs[1, channel].axis('off')

Fig 05 workflow example

In [ ]:
fig, ax = plt.subplots(figsize = (5, 5), dpi = 300, nrows = 1, ncols = 1, subplot_kw = {'projection': crs_4326}, tight_layout = True)

grid = ax.gridlines(draw_labels = True, linestyle = '--', linewidth = 0.5, color = 'gray', crs = crs_4326, dms = True, x_inline = False, y_inline = False)
grid.top_labels = False
grid.right_labels = False

with rio.open(config['Directories']['full_res_dinsar'] / (samples['Amery'] + '.tif')) as tif:
    double_difference = tif.read()
    double_difference = np.swapaxes(double_difference, 0, -1)
    double_difference = np.swapaxes(double_difference, 0, 1)
    
ax.imshow(double_difference, extent = [tif.bounds.left, tif.bounds.right, tif.bounds.bottom, tif.bounds.top])

gl = ais_cci.loc[ais_cci.UUID == samples['Amery']]
gl_4326 = gl.to_crs('EPSG:4326')
gl_4326.plot(ax = ax, color = 'black', label = 'AIS_cci manual GL', legend = True, linewidth = 1)
ax.legend()

In [ ]:
amery_tile = 'SEN_HH_003_D_R_06_20170329222716_20170410222716_20170422222717________________000284_tile_66'

fig1 = vis.plot_preprocessing_example(features_tif = amery_features, gl = gl, tile_tif = config['Directories']['features_stack_tif'] / (amery_tile + '.tif'), basemap = basemap)

with np.load(config['Directories']['nn_outputs'] / 'HED_256_100_wce_rectangular_ensemble' / (amery_tile + '_mu.npz')) as data:
    prediction = data['arr_0'][:, :, 0] 

In [ ]:
with rio.open(config['Directories']['features_stack_tif'] / (amery_tile + '.tif')) as tif:
    transform = tif.transform
fig2 = vis.plot_postprocessing_example(prediction = prediction, transform = transform)

Fig 07, comparing interferometric features

In [ ]:
expts = ['HED_256_100_wce_rectangular', 'HED_256_100_wce_polar']
expts = [config['Directories']['output_vectors'] / expt for expt in expts]

hed_delineation_dfs = Parallel(n_jobs = -1)(delayed(gpd.read_file)(list(expt.glob('*val_loss.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(list(expt.glob('*val_loss_polis_distances.npy'))[0], allow_pickle = True).item()) for expt in expts]
expt_metrics = [np.load(list(expt.glob('*val_loss_metrics.npy'))[0], allow_pickle = True) for expt in expts]

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Getz_1']]
hed_lines = [shp.loc[shp.UUID == samples['Getz_1']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Getz_1']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Getz_1']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig  = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = getz1_features, radarsat_mosaic = radarsat_mosaic, deviations = deviations, coverages = coverages,
                          figure_params = {'size': (11, 5), 'rows': 3, 'columns': 4, 'subplot_pos': [[slice(0, 3, 1), 0], [slice(0, 3, 1), 1]], 'title': ['Rect', 'Pol']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0.01, 0.2, 1, 1),
                                                 'colorbar_width': "20%", 'colorbar_height': "1%", 'colorbar_orientation': 'horizontal',
                                                 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper right', 'bbox': (0.2, 0.2, 0.8, 0.955), 'roi_label': 'Getz Ice Shelf', 
                                                     'width': '55%', 'height': '55%', 'bbox_color': 'red', 'roi_label_loc': 'upper right'},
                          metrics_display_params = {'loc': (0.05, 0.92), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower left', 'size': 5, 'max': 20},
                          zoom_ins = zoom_ins.loc[zoom_ins.ROI == 'Getz_1'],
                          zoom_ins_params = {'col_nums': [1, 3],  'label_pos': [2, 3], 'subplot_pos': [[0, 2], [0, 3], [1, 2], [1, 3], [2, 2], [2, 3]]},
                          shade_subplots = True)

Fig 08, Totten

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Totten']]
hed_lines = [shp.loc[shp.UUID == samples['Totten']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Totten']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Totten']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = totten_features, radarsat_mosaic = radarsat_mosaic, deviations = deviations, coverages = coverages,
                          figure_params = {'size': (10, 6), 'rows': 2, 'columns': 6, 'subplot_pos': [[slice(0, 2, 1), slice(0, 2, 1)], [slice(0, 2, 1), slice(2, 4, 1)]], 'title': ['Rect', 'Pol']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower right', 'bbox': (0, 0.15, 1, 1),
                                                'colorbar_width': "30%", 'colorbar_height': "1%", 'colorbar_orientation': 'horizontal', 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper left', 'bbox': (0, 0, 0.92, 1.17), 'roi_label': 'Totten Ice Shelf', 
                                                     'width': '50%', 'height': '50%', 'bbox_color': 'red',
                                                     'roi_label_loc': 'upper left'},
                          metrics_display_params = {'loc': (0.68, 0.92), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower right', 'size': 5, 'max': 20},
                          zoom_ins = zoom_ins.loc[zoom_ins.ROI == 'Totten'],
                          zoom_ins_params = {'col_nums': [1, 3], 'label_pos': [2, 1], 'subplot_pos': [[0, 4], [0, 5], [1, 4], [1, 5]]})

Fig 09 Spatial vs Temporal dataset, Abbot Ice shelf

In [ ]:
expts = ['HED_256_100_wce_rectangular', 'HED_256_100_wce_rectangular_ood']
expts = [config['Directories']['output_vectors'] / expt for expt in expts]

hed_delineation_dfs = Parallel(n_jobs = -1)(delayed(gpd.read_file)(list(expt.glob('*val_loss.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(list(expt.glob('*val_loss_polis_distances.npy'))[0], allow_pickle = True).item()) for expt in expts]
expt_metrics = [np.load(list(expt.glob('*val_loss_metrics.npy'))[0], allow_pickle = True) for expt in expts]

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Abbot']]
hed_lines = [shp.loc[shp.UUID == samples['Abbot']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Abbot']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Abbot']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = abbot_features, radarsat_mosaic = radarsat_mosaic, deviations = deviations, coverages = coverages,
                          figure_params = {'size': (13, 5), 'rows': 2, 'columns': 6, 'subplot_pos': [[slice(0, 3, 1), slice(0, 2, 1)], [slice(0, 3, 1), slice(2, 4, 1)]], 
                                           'title': ['Rect (temporal dataset)', 'Rect (spatial dataset)']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0.01, 0.035, 1, 1),
                                                 'colorbar_width': "12%", 'colorbar_height': "1%", 'colorbar_orientation': 'horizontal', 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper left', 'bbox': (0, 0, 0.9, 1.05), 'roi_label': 'Abbot Ice Shelf', 'width': '35%', 'height': '35%', 'bbox_color': 'red',
                                                     'roi_label_loc': 'upper right'},
                          metrics_display_params = {'loc': (0.75, 0.9), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower right', 'max': 30},
                          zoom_ins = zoom_ins.loc[zoom_ins.ROI == 'Abbot'],
                          zoom_ins_params = {'col_nums': [1, 3], 'label_pos': [2, 3], 'subplot_pos': [[0, 4], [0, 5], [1, 4], [1, 5]]}) 

Fig 11, Ensemble calc. Amery ice shelf

In [ ]:
ensemble_expts = ['HED_256_100_wce_rectangular_all_seed_0', 'HED_256_100_wce_rectangular_all_seed_42',
                  'HED_256_100_wce_rectangular_all_seed_84', 'HED_256_100_wce_rectangular_all_seed_126',
                  'HED_256_100_wce_rectangular_all_seed_168']

ensemble_stats_tif = config['Directories']['nn_outputs'] / 'HED_256_100_wce_rectangular_all_ensemble' / (samples['Amery'] + '_stats.tif')
ensemble_preds = list((config['Directories']['plot_resources'] / 'ensemble_preds').glob('*.tif'))

ensemble_expts = [config['Directories']['output_vectors'] / expt for expt in ensemble_expts]
expt_metrics = [np.load(list(expt.rglob('metrics.npy'))[0], allow_pickle = True) for expt in ensemble_expts[1:]]
expt_metrics.append(np.load(list(ensemble_expts[0].glob('*val_loss_metrics.npy'))[0], allow_pickle = True))

In [ ]:
fig = vis.plot_ensemble_example(predictions = ensemble_preds, stats_tif = ensemble_stats_tif)

Fig 12, Uncertainty, Amery Ice Shelf

In [ ]:
expts = [config['Directories']['output_vectors'] / expt for expt in ['HED_256_100_wce_rectangular_all_ensemble', 
                                                                     'HED_256_100_wce_rectangular_ensemble']]

ensemble_vectors = Parallel(n_jobs = 1)(delayed(gpd.read_file)(list(expt.glob('*ensemble_mean.geojson'))[0]) for expt in expts)
ensemble_stddev = Parallel(n_jobs = 1)(delayed(gpd.read_file)(list(expt.glob('*stddev_buffer.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(expt / 'polis_distances.npy', allow_pickle = True).item()) for expt in expts]
expt_metrics = [pd.DataFrame(np.load(expt / 'metrics.npy', allow_pickle = True).item()) for expt in expts]
ml_metrics = [np.load(expt / 'ml_metrics.npy', allow_pickle = True) for expt in expts]

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Amery']]
hed_lines = [shp.loc[shp.UUID == samples['Amery']] for shp in ensemble_vectors]
stddev_buffers = [shp.loc[shp.UUID == samples['Amery']] for shp in ensemble_stddev]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Amery']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Amery']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = amery_features, radarsat_mosaic = radarsat_mosaic,
                                         deviations = deviations, coverages = coverages, 
                                         figure_params = {'size': (12, 9), 'rows': 4, 'columns': 5, 'subplot_pos': [[slice(0, 2), slice(0, 3)], [slice(2, 4), slice(0, 3)]],
                                                          'title': ['Rect + Non-Intf ensemble', 'Rect ensemble']},
                                         feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0.01, 0.2, 0.95, 1),
                                                         'colorbar_width': "15%", 'colorbar_height': "1%", 'colorbar_orientation': 'horizontal', 'alpha': 0.6},
                                         antarctica_inset_params = {'loc': 'upper left', 'bbox': (0, 0, 0.6, 1), 'roi_label': 'Amery Ice Shelf', 
                                                                    'width': '40%', 'height': '40%', 'bbox_color': 'red', 'roi_label_loc': 'upper right'},
                                         scalebar_params = {'loc': 'lower left', 'size': 5, 'max': 20},
                                         shade_subplots = True, uncertainty_buffer = stddev_buffers,
                                         metrics_display_params = {'loc': (0.85, 0.87), 'fontsize': 8},
                                         zoom_ins = zoom_ins.loc[zoom_ins.ROI == 'Amery'],
                                         zoom_ins_params = {'label_pos': [2, 3], 'subplot_pos': [[0, 3], [0, 4], [1, 3], [1, 4], [2, 3], [2, 4], [3, 3], [3, 4]]},
                                         uncertainty_buffer_params = {'color': 'maroon', 'alpha': 0.7})

Fig 13, Antarctic-wide deviation

In [ ]:
selected_rois[['ROI']] = ['Fimbulisen', 'Brunt Ice Shelf', 'Getz Ice Shelf', 
                            'Amery Ice Shelf', 'Totten Ice Shelf', 'Ross Ice Shelf', 
                            'Antarctic Peninsula', 'George VI', 'Abbot Ice Shelf', 
                            'Leningradskiy Island', 'Pine Island Glacier', 'Smith Pope Kohler Glaciers', 
                            'Ronne Ice Shelf', 'Filchner Ice Shelf', 
                            'Moscow University Glacier']

In [ ]:
fig = vis.plot_antarctic_wide_deviations_graph(rois = selected_rois, polis_distances = polis_distances, gls = ensemble_vectors, 
                                         feature_subsets = ['Rectangular + non_interferometric', 'Rectangular'])

Appendix figures

Fig. A1

In [ ]:
raster_orig = config['Directories']['work_dir'].parent / 'qgis_resources/appendix_resampling_example' / (samples['Ketchum_Glacier'] + '.tif')
raster_interferometric_features = config['Directories']['work_dir'].parent / 'qgis_resources/appendix_resampling_example' / (samples['Ketchum_Glacier'] + '_4326_interferometric.tif')

fig = vis.plot_features_4326(raster_orig, raster_interferometric_features, inset_params = {'loc': 'upper right', 'bbox': (0, 0, 1, 1), 'roi_label': '', 
                                                                    'width': '40%', 'height': '40%', 'bbox_color': 'red', 'roi_label_loc': 'lower right'}, figparams = {'figsize': (10, 10), 'dpi': 300})

Fig. A2

In [ ]:
fig = vis.plot_resampled_features(raster_3031 = config['Directories']['downsampled_dinsar'] / (samples['Ketchum_Glacier'] + '.tif'), insets = gpd.read_file('rois/phase_resampling_insets.shp'), figparams = {'figsize': (10, 10), 'dpi': 300})

Fig. B1 feature subsets

In [ ]:
expts = ['HED_256_100_wce_rectangular', 'HED_256_100_wce_polar', 'HED_256_100_wce_rectangular_all_seed_0']
expts = [config['Directories']['output_vectors'] / expt for expt in expts]

hed_delineation_dfs = Parallel(n_jobs = -1)(delayed(gpd.read_file)(list(expt.glob('*val_loss.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(list(expt.glob('*val_loss_polis_distances.npy'))[0], allow_pickle = True).item()) for expt in expts]

Fig B2 feature subsets

In [ ]:
expts = ['HED_256_100_wce_rectangular_dem', 'HED_256_100_wce_rectangular_iv', 'HED_256_100_wce_rectangular_tides']
expts = [config['Directories']['output_vectors'] / expt for expt in expts]

hed_delineation_dfs = Parallel(n_jobs = -1)(delayed(gpd.read_file)(list(expt.glob('*val_loss.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(list(expt.glob('*val_loss_polis_distances.npy'))[0], allow_pickle = True).item()) for expt in expts]

Fig B3 feature subsets

In [ ]:
expts = ['HED_256_100_wce_rectangular_iv_tides', 'HED_256_100_wce_rectangular_dem_tides', 'HED_256_100_wce_rectangular_dem_iv']
expts = [config['Directories']['output_vectors'] / expt for expt in expts]

hed_delineation_dfs = Parallel(n_jobs = -1)(delayed(gpd.read_file)(list(expt.glob('*val_loss.geojson'))[0]) for expt in expts)
polis_distances = [pd.DataFrame(np.load(list(expt.glob('*val_loss_polis_distances.npy'))[0], allow_pickle = True).item()) for expt in expts]

George VI Ice shelf

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['George_VI']]
hed_lines = [shp.loc[shp.UUID == samples['George_VI']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['George_VI']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['George_VI']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = george_vi, radarsat_mosaic = radarsat_mosaic, deviations = deviations, coverages = coverages,
                          figure_params = {'size': (15, 8), 'rows': 1, 'columns': 3, 'subplot_pos': [[0, 0], [0, 1], [0, 2]], "title": ['Rect + Vel + DEM', 'Rect + DEM + Tide', 'Rect + DEM + Vel']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0, 0.1, 0.8, 0.9),
                                                 'colorbar_width': "15%", 'colorbar_height': "2%", 'colorbar_orientation': 'horizontal',
                                                 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper left', 'bbox': (0, 0.2, 0.5, 0.8), 'roi_label': 'George VI Ice Shelf', 
                                                     'width': '78%', 'height': '78%', 'bbox_color': 'red'},
                          metrics_display_params = {'loc': (0.78, 0.81), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower right', 'size': 5, 'max': 20},
                          shade_subplots = True)
fig.savefig('/mnt/local/Sindhu/paper1_figures/appendix_figs/b3_george.png', dpi = 300, bbox_inches = 'tight')

Getz 2

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Getz_2']]
hed_lines = [shp.loc[shp.UUID == samples['Getz_2']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Getz_2']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Getz_2']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig  = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = getz2_features, radarsat_mosaic = radarsat_mosaic, deviations = deviations, coverages = coverages,
                                          figure_params = {'size': (15, 8), 'rows': 1, 'columns': 3, 'subplot_pos': [[0, 0], [0, 1], [0, 2]], 'title': ['', '', '']},
                                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0.008, 0.1, 1, 1),
                                                             'colorbar_width': "20%", 'colorbar_height': "1%", 'colorbar_orientation': 'horizontal',
                                                              'alpha': 0.6},
                                          antarctica_inset_params = {'loc': 'upper right', 'bbox': (0.2, 0.2, 0.8, 0.88), 'roi_label': 'Getz Ice Shelf', 
                                                     'width': '45%', 'height': '45%', 'bbox_color': 'red', 'roi_label_loc': 'bottom left'},
                                          metrics_display_params = {'loc': (0.03, 0.95), 'fontsize': 8},
                                          scalebar_params = {'loc': 'lower left', 'size': 5, 'max': 20},
                                          shade_subplots = True)
fig.savefig('/mnt/local/Sindhu/paper1_figures/appendix_figs/b3_getz.png', dpi = 300, bbox_inches = 'tight')

Leningradskiy Island

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['Lazarevisen']]
hed_lines = [shp.loc[shp.UUID == samples['Lazarevisen']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Lazarevisen']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['Lazarevisen']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = lazarevisen_features, radarsat_mosaic = radarsat_mosaic, 
                                         deviations = deviations, coverages = coverages,
                          figure_params = {'size': (15, 8), 'rows': 1, 'columns': 3, 'subplot_pos': [[0, 0], [0, 1], [0, 2]], 'title': ['', '', '']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower right', 'bbox': (0, 0.3, 1, 0.9),
                                                 'colorbar_width': "15%", 'colorbar_height': "2%", 'colorbar_orientation': 'horizontal',
                                                 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper left', 'bbox': (0.01, 0, 0.5, 1), 'roi_label': 'Leningradskiy Island', 
                                                     'width': '60%', 'height': '60%', 'bbox_color': 'red'},
                          metrics_display_params = {'loc': (0.81, 0.85), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower right', 'size': 5, 'max': 20},
                          shade_subplots = True)
fig.savefig('/mnt/local/Sindhu/paper1_figures/appendix_figs/b2_lazarevisen.png', dpi = 300, bbox_inches = 'tight')

Larsen C

In [ ]:
gt = ais_cci.loc[ais_cci.UUID == samples['LarsenC']]
hed_lines = [shp.loc[shp.UUID == samples['LarsenC']] for shp in hed_delineation_dfs]
deviations = [np.nanmean(line_metric.loc[line_metric.UUID == samples['LarsenC']].deviations.values[0].astype(np.float64)) for line_metric in polis_distances]
coverages = [np.nanmean(line_metric.loc[line_metric.UUID == samples['LarsenC']].coverage.values[0].astype(np.float64)) for line_metric in polis_distances]

In [ ]:
fig = vis.overlay_gls_on_dinsar_subplots(manual_line = gt, pred_lines = hed_lines, features_stack = larsenc_features, radarsat_mosaic = radarsat_mosaic, 
                                         deviations = deviations, coverages = coverages,
                          figure_params = {'size': (15, 8), 'rows': 1, 'columns': 3, 'subplot_pos': [[0, 0], [0, 1], [0, 2]], 'title': ['', '', '']},
                          feature_plot_params = {'colorbar_label': 'radians', 'cmap': 'hsv', 'colorbar_loc': 'lower left', 'bbox': (0, 0.1, 1, 0.9),
                                                 'colorbar_width': "15%", 'colorbar_height': "2%", 'colorbar_orientation': 'horizontal',
                                                 'alpha': 0.6},
                          antarctica_inset_params = {'loc': 'upper left', 'bbox': (0.01, 0, 0.5, 1), 'roi_label': 'Larsen C Ice Shelf', 
                                                     'width': '58%', 'height': '58%', 'bbox_color': 'red'},
                          metrics_display_params = {'loc': (0.76, 0.78), 'fontsize': 8},
                          scalebar_params = {'loc': 'lower right', 'size': 5, 'max': 20},
                          shade_subplots = True)
fig.savefig('/mnt/local/Sindhu/paper1_figures/appendix_figs/b3_larsenc.png', dpi = 300, bbox_inches = 'tight')